In [1]:
from scipy.spatial import distance
from imutils import face_utils
import imutils
import dlib
import cv2

In [2]:
def eye_aspect_ratio(eye):
	A = distance.euclidean(eye[1], eye[5])
	B = distance.euclidean(eye[2], eye[4])
	C = distance.euclidean(eye[0], eye[3])
	ear = (A + B) / (2.0 * C)
	return ear


In [3]:
def mouth_aspect_ratio(muh):
    A = distance.euclidean(muh[13], muh[19])
    B = distance.euclidean(muh[15], muh[17])
    C = distance.euclidean(muh[12], muh[16])
    ear1 = (A + B) / (2.0 * C)
    return ear1

In [10]:
threshEye = 0.32
threshLip = 0.3
frame_check = 30
detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
(uStart, uEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

In [11]:
cap=cv2.VideoCapture(0)
flag=0
global lipear
global ear
while True:
    ret, frame=cap.read()
    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  #  gray = light_removing(frame)
    subjects = detect(gray, 0)
    for subject in subjects:
        shape = predict(gray, subject)
        shape = face_utils.shape_to_np(shape)#converting to NumPy Array
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        lips = shape[uStart:uEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        lipear = mouth_aspect_ratio(lips)
        ear = (leftEAR + rightEAR) / 2.0
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        lipHull = cv2.convexHull(lips)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [lipHull], -1, (0, 255, 0), 1)
        if ear < threshEye or lipear > threshLip:
            flag += 1
            #print (flag)
            if flag >= frame_check:
                if lipear > threshLip and ear < threshEye:
                    cv2.putText(frame, "Yawn", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    cv2.putText(frame, "Eyes", (10,325),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                elif ear < threshEye: 
                    cv2.putText(frame, "Eyes", (10,325),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                elif lipear > threshLip:
                    cv2.putText(frame, "Yawn", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    
        else:
            flag = 0
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        cv2.destroyAllWindows()
        cap.release()
        break